 ## 変数クラスの試作

 ここのコメント文はMarkdown形式で保存できます。

In [126]:
import numpy as np

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [127]:
# 変数クラスの定義
# data,grad: NumPyの多次元配列
#
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [128]:
# 関数クラスの定義
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    # gy : 出力側から伝わる微分
    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def exp(x):
    return Exp()(x)

In [129]:
# numpy.arrayで変数オブジェクトを作成する
data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [130]:
# 変数オブジェクトxに新しいデータを代入する
x.data = np.array(2.0)
print(x.data)

2.0


In [131]:
# numpy多次元配列
x1 = np.array(1)
print(x1.ndim)

x2 = np.array([1,2,3])
print(x2.ndim)

x3 = np.array([[1,2,3],
               [4,5,6]])
print(x3.ndim)

0
1
2


In [132]:
#
x = Variable(np.array(10))
y = square(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [133]:
# 数値微分関数の定義(中心差分近似)
# f(x+h)-f(x-h)/2h 
# le-4=0.0001
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(as_array(x.data - eps))
    x1 = Variable(as_array(x.data + eps))
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f,x)
print(dy)

4.000000000004


In [134]:
# 関数連鎖による順伝搬と逆伝搬の例
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)
print(y.data)

#逆向きに計算グラフのノードを辿る
assert isinstance(y.creator, Square)
assert y.creator.input == b
assert isinstance(y.creator.input.creator, Exp)
assert y.creator.input.creator.input == a
assert isinstance(y.creator.input.creator.input.creator, Square)
assert y.creator.input.creator.input.creator.input == x

y.backward()
print(x.grad)

# 数値微分との比較
def f(x):
    return square(exp(square(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

x = Variable(None)
try:
    x = Variable(1.0) # NG:エラーが発生
except TypeError as e:
    print(e)

1.648721270700128
3.297442541400256
3.2974426293330694
<class 'float'> is not supported
